<a href="https://colab.research.google.com/github/lmlima/PoolAttendance/blob/master/Apresenta%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Dependências

- Explicar o uso de cada biblioteca

In [1]:
#@title Instalando dependências
#install TensorFlow 2.0 alpha version
!pip install tensorflow==2.0.0-alpha0

# Load the TensorBoard notebook extension
%load_ext tensorboard.notebook

# multivariate one step problem with lstm
import tensorflow as tf
import os
import datetime

from numpy import array
from numpy import hstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

     |████████████████████████████████| 79.9MB 39kB/s 
     |████████████████████████████████| 3.0MB 27.5MB/s 
     |████████████████████████████████| 419kB 37.7MB/s 
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

# 2. Base de dados

## 2.1. Define a base de dados

- Apresentar e fazer uma breve analise da base de dados

In [0]:
# define dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])

# reshape series
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2))
target = in_seq2

## 2.2. Prepara os dados para a predição

- Explicar as etapas da preparação e seus parâmetros

In [3]:
# define generator
n_features = dataset.shape[1]
n_output = target.shape[1]

window_size = 2
n_batch = 8

generator = TimeseriesGenerator(dataset, target, length=window_size, batch_size=n_batch)

# Visualização
print(F"Número de características de entrada: {n_features}")
print(F"Número de características de saída: {n_output}\n")

print(F"Janela de avaliação: {window_size}")
print(F"Tamanho do batch: {n_batch}")

Número de características de entrada: 2
Número de características de saída: 1

Janela de avaliação: 2
Tamanho do batch: 8


# 3. Modelo



## 3.1. Arquitetura do modelo
- Apresentar as diferentes alternativas
- Explicar qual utilizar e pq

In [0]:
# define model
def baseline_model(): 
  model = Sequential()
  model.add(LSTM(100, activation='relu', input_shape=(window_size, n_features)))
  model.add(Dense(n_output))
  model.compile(optimizer='adam', loss='mse', metrics=['mae', tf.keras.metrics.RootMeanSquaredError()])
  return model

b_model = baseline_model()

## 3.2. Treinamento
- Apresentar os hiperparâmetros e pq
- Visualizar treinamento

In [0]:
# Configuração do Tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [6]:
# fit model
n_epochs = 100
n_steps = 1

b_model.fit_generator(generator, steps_per_epoch=n_steps, epochs=n_epochs, callbacks=[tensorboard_callback], verbose=1)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 4404.6543 - mae: 63.4837 - root_mean_squared_error: 66.3676
Epoch 2/100
1/1 [==============================] - 0s 102ms/step - loss: 4245.9170 - mae: 62.3814 - root_mean_squared_error: 65.1607
Epoch 3/100
1/1 [==============================] - 0s 13ms/step - loss: 4091.4761 - mae: 61.2881 - root_mean_squared_error: 63.9646
Epoch 4/100
1/1 [==============================] - 0s 85ms/step - loss: 3940.3540 - mae: 60.1967 - root_mean_squared_error: 62.7722
Epoch 5/100
1/1 [==============================] - 0s 96ms/step - loss: 3792.0361 - mae: 59.1040 - root_mean_squared_error: 61.5795
Epoch 6/100
1/1 [==============================] - 0s 96ms/step - loss: 3646.2212 - mae: 58.0076 - root_mean_squared_error: 60.3839
Epoch 7/100
1/1 [==============================] - 0s 97ms/step - loss: 3502.5103 - mae: 56.9041 - root_mean_squared_error: 59.1820
Epoch 8/100
1/1 [==============================] - 0s 96ms/step - loss: 3360.

In [7]:
#@title Visualizar treinamento
%tensorboard --logdir logs

## 3.3. Teste
- Apresentar teste de validação estatistica
- Apresentar uso ilustrativo do modelo

In [8]:
# make a one step prediction out of sample
x_input = array([[90, 95], [100, 105]]).reshape((1, window_size, n_features))
yhat = b_model.predict(x_input, verbose=0)
print(yhat)


[[117.31599]]
